In [162]:
import pandas as pd
from IPython.display import display
import mysql.connector
import os
from dotenv import load_dotenv
import pymysql
from sqlalchemy import create_engine
import firebase_admin
from firebase_admin import credentials, storage
from datetime import datetime
from google.cloud import bigquery
import numpy as np
import mimetypes
from io import StringIO

In [163]:
load_dotenv()

True

In [164]:
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

capstone-de2-3b8d363c4eab.json


# Extract Data

In [165]:
def ingestion(table):
    load_dotenv()
    connection = mysql.connector.connect(
        host=os.getenv('HOST'),
        user='root',
        password=os.getenv('PASSWORD'),
        database='capstone5'
    )
    query = f"SELECT * FROM {table}"  
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [166]:
# List of table names
tables = [
    'applications', 'articles', 'comments', 'donation_manual_comments',
    'donation_manuals', 'fundraising_categories', 'fundraisings',
    'likes_comments', 'organizations', 'testimoni_volunteers',
    'user_bookmark_fundraisings', 'user_bookmark_volunteer_vacancies',
    'volunteers', 'users', 'admins', 'like_donation_comments',
    'user_bookmark_articles', 'user_bookmark_articles'
]

# Ingest data for all tables
data_frame = [ingestion(table) for table in tables]

C:\Users\SINTA\AppData\Local\Temp\ipykernel_23304\4175512421.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\SINTA\AppData\Local\Temp\ipykernel_23304\4175512421.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


# Load Raw Data To Firebase

In [167]:
def initialize_firebase():
    if not firebase_admin._apps:
        credentials_path = os.getenv('FIREBASE_CREDENTIALS_PATH')
        cred = credentials.Certificate(credentials_path)
        firebase_admin.initialize_app(cred)
        print("Firebase has been initialized")
    else:
        print("Firebase is already initialized")
    bucket_name = os.getenv('BUCKET_NAME')
    return storage.bucket(bucket_name)

In [168]:
def create_folder_in_bucket():
    bucket = initialize_firebase()
    current_date = datetime.now().strftime("%Y%m%d")
    folder_blob = bucket.blob(f"{current_date}/")
    folder_blob.upload_from_string('')
    print(f"Folder '{current_date}' created successfully.")
    return current_date

In [169]:
def upload_dataframes_to_firebase(data_frames):
    current_date = create_folder_in_bucket()
    bucket = initialize_firebase()
    
    for df, table_name in zip(data_frames, tables):
        # Convert dataframe to CSV string
        csv_str = df.to_csv(index=False)

        # Create the blob reference with folder name
        file_name_with_date = f"{table_name}_{current_date}.csv"
        file_path_in_bucket = f"{current_date}/{file_name_with_date}"
        file_ref = bucket.blob(file_path_in_bucket)
        
        # Upload CSV string to Firebase
        file_ref.upload_from_string(csv_str, content_type='text/csv')
        print(f"Dataframe {table_name} uploaded successfully as {file_name_with_date}!")

In [170]:
# Upload all dataframes to Firebase
upload_dataframes_to_firebase(data_frame)

Firebase is already initialized
Folder '20240621' created successfully.
Firebase is already initialized
Dataframe applications uploaded successfully as applications_20240621.csv!
Dataframe articles uploaded successfully as articles_20240621.csv!
Dataframe comments uploaded successfully as comments_20240621.csv!
Dataframe donation_manual_comments uploaded successfully as donation_manual_comments_20240621.csv!
Dataframe donation_manuals uploaded successfully as donation_manuals_20240621.csv!
Dataframe fundraising_categories uploaded successfully as fundraising_categories_20240621.csv!
Dataframe fundraisings uploaded successfully as fundraisings_20240621.csv!
Dataframe likes_comments uploaded successfully as likes_comments_20240621.csv!
Dataframe organizations uploaded successfully as organizations_20240621.csv!
Dataframe testimoni_volunteers uploaded successfully as testimoni_volunteers_20240621.csv!
Dataframe user_bookmark_fundraisings uploaded successfully as user_bookmark_fundraisings

# Transform

### Handle Missing Value & Duplicates

In [171]:
# Cek duplikat dan missing value

def cleaning_data(df):
    
    # cek duplikat
    duplicates = df[df.duplicated(subset=df.columns, keep=False)]
    if not duplicates.empty:
        print(f"Terdapat duplikat pada {df}")
        print(duplicates)
        df = df.drop_duplicates()
    else:
        print(f"Tidak ada data duplikat pada {df}")
        
    # Ubah tipe data
    columns_datetime = ['created_at', 'updated_at', 'deleted_at']
    for col in columns_datetime:
        if df[col].dtype != 'datetime64[ns]':
            df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
        df[col] = df[col].dt.strftime('%Y-%m-%d')
    

    # cek missing value
    missing_columns = [col for col in df.columns if col not in ['updated_at', 'deleted_at']]
    # df[missing_columns] = df[missing_columns].replace('', None)
    if df[missing_columns].isnull().any().any():
        print(f"Terdapat missing value dalam {df}")
        missing_sum = df[missing_columns].isnull().sum()
        print(missing_sum)
        
        columns_numeric = ['total_likes', 'amount', 'goal_amount', 'current_progress', 'is_verified', 'registered_volunteer']
        for col in missing_columns:
            if df[col].dtype != 'object' and df[col].dtype != 'datetime64[ns]' and col not in columns_numeric:
                df[col] = df[col].fillna(np.nan)
            elif col in columns_numeric :
                df[col] = df[col].fillna(0)
            elif df[col].dtype == 'datetime64[ns]':
                df[col] = df[col].fillna(pd.NaT)
            else:
                df[col] = df[col].fillna('Unknown')
    else :
        print(f"Tidak ada missing value dalam {df}")
        missing_sum = df[missing_columns].isnull().sum()
        print(missing_sum)
    return df

In [172]:
for df in data_frame :
    cleaning_data(df)

Tidak ada data duplikat pada       id              created_at              updated_at deleted_at  \
0      1 2024-06-07 13:08:19.244 2024-06-07 13:08:19.244       None   
1      4 2024-06-08 13:08:19.244                     NaT       None   
2      5 2024-06-08 13:08:19.244                     NaT       None   
3      6 2024-06-08 13:08:19.244                     NaT       None   
4      7 2024-06-08 13:08:19.244                     NaT       None   
..   ...                     ...                     ...        ...   
144  190 2024-06-19 15:29:56.881 2024-06-19 15:29:56.881       None   
145  191 2024-06-19 15:34:24.573 2024-06-19 15:34:24.573       None   
146  192 2024-06-19 16:22:30.107 2024-06-19 16:22:30.107       None   
147  193 2024-06-19 19:13:35.111 2024-06-19 19:13:35.111       None   
148  194 2024-06-20 06:20:52.239 2024-06-20 06:20:52.239       None   

                                          ig_image_url  \
0    https://res.cloudinary.com/dvrhf8d9t/image/upl...   
1 

In [173]:
df_applications = data_frame[0]
df_articles = data_frame[1]
# df_comments = data_frame[2]
# df_donation_manual_comments = data_frame[3]
df_donation_manuals = data_frame[4]
df_fundraising_categories = data_frame[5]
df_fundraisings = data_frame[6]
# df_like_comments = data_frame[7]
df_organizations = data_frame[8]
df_testimoni_volunteers = data_frame[9]
df_user_bookmark_fundraisings = data_frame[10]
df_user_bookmark_volunteer_vacancies = data_frame[11]
df_volunteers = data_frame[12]
df_users = data_frame[13]
# df_admins = data_frame[14]
# df_like_donation_comments = data_frame[15]
df_user_bookmark_articles = data_frame[16]

In [174]:
df_applications

,id,created_at,updated_at,deleted_at,ig_image_url,yt_image_url,user_id,vacancy_id,job,reason,age
0,1,2024-06-07,2024-06-07,NaN,https://res.cloudinary.com/dvrhf8d9t/image/upl...,https://res.cloudinary.com/dvrhf8d9t/image/upl...,54,3,Mahasiswa,Ingin menambah produktivitas,23.0
1,4,2024-06-08,NaN,NaN,https://res.cloudinary.com/dvrhf8d9t/image/upl...,https://res.cloudinary.com/dvrhf8d9t/image/upl...,54,1,Mahasiswa,Ingin menambah produktivitas,21.0
2,5,2024-06-08,NaN,NaN,https://res.cloudinary.com/dvrhf8d9t/image/upl...,https://res.cloudinary.com/dvrhf8d9t/image/upl...,55,1,Mahasiswa,Ingin menambah relasi,NaN
3,6,2024-06-08,NaN,NaN,https://res.cloudinary.com/dvrhf8d9t/image/upl...,https://res.cloudinary.com/dvrhf8d9t/image/upl...,56,1,Mahasiswa,Ingin mengisi waktu,NaN
4,7,2024-06-08,NaN,NaN,https://res.cloudinary.com/dvrhf8d9t/image/upl...,https://res.cloudinary.com/dvrhf8d9t/image/upl...,58,1,Mahasiswa,Ingin menambah pengalaman,NaN
...,...,...,...,...,...,...,...,...,...,...,...
144,190,2024-06-19,2024-06-19,NaN,https://res.cloudinary.com/dvrhf8d9t/image/upl...,https://res.cloudinary.com/dvrhf8d9t/image/upl...,127,3,Karyawan Swasta,testt,23.0
145,191,2024-06-19,2024-06-19,NaN,https://res.cloudinary.com/dvrhf8d9t/image/upl...,https://res.cloudinary.com/dvrhf8d9t/image/upl...,128,4,Karyawan Swasta,asdsa,19.0
146,192,2024-06-19,2024-06-19,NaN,https://res.cloudinary.com/dvrhf8d9t/image/upl...,https://res.cloudinary.com/dvrhf8d9t/image/upl...,127,6,Mahasiswa,test,19.0
147,193,2024-06-19,2024-06-19,NaN,https://res.cloudinary.com/dvrhf8d9t/image/upl...,https://res.cloudinary.com/dvrhf8d9t/image/upl...,121,3,Mahasiswa,fomo,19.0


### Tabel Fakta

##### 1. FactDonationTransaction

In [175]:
# Buat Struktur kolom fact_donation_transaction
columns = ['id', 'donation_id', 'fundraising_id', 'user_id', 'amount', 'goal_amount', 'fundraising_category_id', 'organization_id', 'created_at']
fact_donation = pd.DataFrame(columns=columns)

# mengambil data yang sukses di df_donation 
df_donation_success = df_donation_manuals.loc[df_donation_manuals['status'] == 'sukses']
df_donation_success = df_donation_success.reset_index(drop=True)

# mengisi data pada dari kolom df_donation
fact_donation['id'] = range(1, len(df_donation_success) + 1)
fact_donation['donation_id'] = df_donation_success['id']
fact_donation['fundraising_id'] = df_donation_success['fundraising_id']
fact_donation['user_id'] = df_donation_success['user_id']
fact_donation['amount'] = df_donation_success['amount']
fact_donation['created_at'] = df_donation_success['created_at']

# merge df_fundraising
df_merge_fact_fundraising = pd.merge(fact_donation, df_fundraisings, left_on='fundraising_id', right_on='id', how='left')
fact_donation['goal_amount'] = df_merge_fact_fundraising['goal_amount_y']
fact_donation['fundraising_category_id'] = df_merge_fact_fundraising['fundraising_category_id_y']
fact_donation['organization_id'] = df_merge_fact_fundraising['organization_id_y']

In [176]:
fact_donation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       100 non-null    int64 
 1   donation_id              100 non-null    int64 
 2   fundraising_id           100 non-null    int64 
 3   user_id                  100 non-null    int64 
 4   amount                   100 non-null    int64 
 5   goal_amount              100 non-null    int64 
 6   fundraising_category_id  100 non-null    int64 
 7   organization_id          100 non-null    int64 
 8   created_at               100 non-null    object
dtypes: int64(8), object(1)
memory usage: 7.2+ KB


##### 2. FactVolunteerApplications

In [177]:
# Buat Struktur kolom df_fact_volunteer_applications
columns = ['id', 'application_id', 'vacancy_id', 'user_id', 'organization_id', 'created_at']
fact_applications = pd.DataFrame(columns=columns)

# mengisi data pada dari kolom df_application
fact_applications['id'] = range(1, len(df_applications) + 1)
fact_applications['application_id'] = df_applications['id']
fact_applications['vacancy_id'] = df_applications['vacancy_id']
fact_applications['user_id'] = df_applications['user_id']
fact_applications['created_at'] = df_applications['created_at']

# merge df_fundraising
df_merge_fact_volunteer = pd.merge(fact_applications, df_volunteers, left_on='vacancy_id', right_on='id', how='left')
fact_applications['organization_id'] = df_merge_fact_volunteer['organization_id_y']

In [178]:
fact_applications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               149 non-null    int64 
 1   application_id   149 non-null    int64 
 2   vacancy_id       149 non-null    int64 
 3   user_id          149 non-null    int64 
 4   organization_id  149 non-null    int64 
 5   created_at       149 non-null    object
dtypes: int64(5), object(1)
memory usage: 7.1+ KB


##### 3. FactVolunteerTestimoni

In [179]:
# Buat Struktur kolom df_fact_volunteer_testimoni
columns = ['id', 'user_id', 'vacancy_id', 'testimoni_volunteer_id', 'rating', 'created_at']
fact_volunteer_testimoni = pd.DataFrame(columns=columns)

# mengisi data pada dari kolom df_application
fact_volunteer_testimoni['id'] = range(1, len(df_testimoni_volunteers) + 1)
fact_volunteer_testimoni['user_id'] = df_testimoni_volunteers['user_id']
fact_volunteer_testimoni['vacancy_id'] = df_testimoni_volunteers['vacancy_id']
fact_volunteer_testimoni['testimoni_volunteer_id'] = df_testimoni_volunteers['id']
fact_volunteer_testimoni['rating'] = df_testimoni_volunteers['rating']
fact_volunteer_testimoni['created_at'] = df_testimoni_volunteers['created_at']

##### 4. FactArticlePopular

In [180]:
# Buat Struktur kolom df_fact_articel_popular
columns = ['id', 'article_id', 'bookmark_id', 'user_id', 'created_at']
fact_article_popular = pd.DataFrame(columns=columns)

# mengisi data pada dari kolom df_comment
fact_article_popular['id'] = range(1, len(df_user_bookmark_articles) + 1)
fact_article_popular['article_id'] = df_user_bookmark_articles['article_id']
fact_article_popular['bookmark_id'] = df_user_bookmark_articles['id']
fact_article_popular['user_id'] = df_user_bookmark_articles['user_id']
fact_article_popular['created_at'] = df_user_bookmark_articles['created_at']

##### 5. FactBookmarkFundraising

In [181]:
fact_bookmark_fundraising = df_user_bookmark_fundraisings.drop(['deleted_at', 'updated_at'], axis=1)
fact_bookmark_fundraising = fact_bookmark_fundraising.rename(columns={'id':'bookmark_id'})
fact_bookmark_fundraising['id'] = range(1, len(fact_bookmark_fundraising) + 1)
fact_bookmark_fundraising.insert(0, 'id', fact_bookmark_fundraising.pop('id'))

##### 6. FactBookmarkVolunteerVacancies

In [182]:
fact_bookmark_volunteer_vacancies = df_user_bookmark_volunteer_vacancies.drop(['deleted_at', 'updated_at','volunteer_vacancy_id'], axis=1)
fact_bookmark_volunteer_vacancies = fact_bookmark_volunteer_vacancies.rename(columns={'id':'bookmark_id'})
fact_bookmark_volunteer_vacancies['id'] = range(1, len(fact_bookmark_volunteer_vacancies) + 1)
fact_bookmark_volunteer_vacancies.insert(0, 'id', fact_bookmark_volunteer_vacancies.pop('id'))

### Tabel Dimensi

In [183]:
dim_fundraisings = df_fundraisings.drop(['fundraising_category_id','organization_id','updated_at'], axis=1)
dim_fundraising_categories = df_fundraising_categories[['id','name','created_at']]
dim_donation_manual = df_donation_manuals.drop(['fundraising_id', 'user_id','updated_at'], axis=1)
dim_organization = df_organizations.drop(['updated_at'], axis= 1)
dim_user = df_users.drop(['updated_at'], axis=1)
dim_volunteer_applictaion = df_applications.drop(['user_id','vacancy_id','updated_at'], axis=1)
dim_volunteer_vacancies = df_volunteers.drop(['organization_id','updated_at'], axis=1)
dim_testimoni_volunteer = df_testimoni_volunteers.drop(['user_id','vacancy_id','updated_at'], axis=1)
dim_article = df_articles.drop(['updated_at'], axis = 1)
# dim_comment = df_comments.drop(['user_id','article_id','updated_at'], axis=1)
dim_bookmark_fundraising = df_user_bookmark_fundraisings.drop(['fundraising_id','user_id','updated_at'], axis=1)
dim_bookmark_volunter_vacancies = df_user_bookmark_volunteer_vacancies.drop(['volunteer_vacancies_id','user_id','updated_at'], axis=1)
dim_bookmark_article = df_user_bookmark_articles.drop(['deleted_at','updated_at'], axis=1)

In [184]:
df_fact = [
    ('fact_applications', fact_applications),
    ('fact_article_popular', fact_article_popular),
    ('fact_bookmark_fundraising', fact_bookmark_fundraising),
    ('fact_bookmark_volunteer_vacancies', fact_bookmark_volunteer_vacancies),
    ('fact_donation', fact_donation),
    ('fact_volunteer_testimoni', fact_volunteer_testimoni)
]

df_dim = [
    ('dim_article', dim_article),
    ('dim_bookmark_fundraising', dim_bookmark_fundraising),
    ('dim_bookmark_volunter_vacancies', dim_bookmark_volunter_vacancies),
    ('dim_bookmark_article', dim_bookmark_article),
    ('dim_donation_manual', dim_donation_manual),
    ('dim_fundraising_categories', dim_fundraising_categories),
    ('dim_fundraisings', dim_fundraisings),
    ('dim_organization', dim_organization),
    ('dim_testimoni_volunteer', dim_testimoni_volunteer),
    ('dim_user', dim_user),
    ('dim_volunteer_application', dim_volunteer_applictaion),
    ('dim_volunteer_vacancies', dim_volunteer_vacancies)
]

# Load

### Load To DB Local

In [185]:
# def load_db_local(df, table_name):
#     connection = pymysql.connect(
#         host='localhost',
#         user='root',
#         password='',
#         port=int(os.getenv('port'))
#     )

#     cursor = connection.cursor()

#     cursor.execute('CREATE DATABASE IF NOT EXISTS peduli_pintar')
#     cursor.execute('USE peduli_pintar')

#     engine = create_engine('mysql+pymysql://root:@localhost:3307/peduli_pintar')
#     df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
#     connection.close()

In [186]:
# load_db_local(df_applications,'tes')

In [187]:
# # Load dimension tables
# for table_name, df in df_dim:
#     load_db_local(df, table_name)

# for table_name, df in df_fact:
#     load_db_local(df, table_name)

### Load To Bigquery

In [188]:
credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if credentials_path:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path
else:
    raise Exception("GOOGLE_APPLICATION_CREDENTIALS is not set in the .env file")

client = bigquery.Client()

dataset_id_fact = os.getenv("dataset_id_fact")
dataset_id_dim = os.getenv("dataset_id_dim")

In [189]:
def load_df_to_bigquery(dataset_id, table_name, df):
    # Create a BigQuery client
    client = bigquery.Client()

    # Convert DataFrame to CSV
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    csv_buffer.seek(0)

    table_id = f"{dataset_id}.{table_name}"
    
    partition_by = bigquery.TimePartitioning(field="created_at")

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        autodetect=True,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
        time_partitioning=partition_by
    )

    # Load CSV data from StringIO buffer
    job = client.load_table_from_file(csv_buffer, table_id, job_config=job_config)
    
    # Wait for the load job to complete
    job.result()

    # Get table information
    table = client.get_table(table_id)
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

In [190]:
# Load fact tables
for table_name, df in df_fact:
    load_df_to_bigquery(dataset_id_fact, table_name, df)

# Load dimension tables
for table_name, df in df_dim:
    load_df_to_bigquery(dataset_id_dim, table_name, df)

Loaded 149 rows and 6 columns to peduli_pintar_fact.fact_applications
Loaded 52 rows and 5 columns to peduli_pintar_fact.fact_article_popular
Loaded 94 rows and 5 columns to peduli_pintar_fact.fact_bookmark_fundraising
Loaded 67 rows and 5 columns to peduli_pintar_fact.fact_bookmark_volunteer_vacancies
Loaded 100 rows and 9 columns to peduli_pintar_fact.fact_donation
Loaded 24 rows and 6 columns to peduli_pintar_fact.fact_volunteer_testimoni
Loaded 43 rows and 7 columns to peduli_pintar_dim.dim_article
Loaded 94 rows and 3 columns to peduli_pintar_dim.dim_bookmark_fundraising
Loaded 67 rows and 4 columns to peduli_pintar_dim.dim_bookmark_volunter_vacancies
Loaded 52 rows and 4 columns to peduli_pintar_dim.dim_bookmark_article
Loaded 164 rows and 6 columns to peduli_pintar_dim.dim_donation_manual
Loaded 4 rows and 3 columns to peduli_pintar_dim.dim_fundraising_categories
Loaded 67 rows and 11 columns to peduli_pintar_dim.dim_fundraisings
Loaded 22 rows and 12 columns to peduli_pintar_di